# 🚨 Katastrophen-Tweets: NLP Master-Projekt
### Automatisierte Analyse, Modellierung und Benchmarking

Dieses Notebook demonstriert den Einsatz der zentralen Bibliothek `nlp_utils.py`, um eine vollständige NLP-Pipeline aufzubauen.
Es umfasst:
1.  **Daten laden & EDA**: Verteilungen, Wordclouds, N-Gramme.
2.  **Modell-Benchmark**: Ein Wettkampf zwischen 25+ Algorithmen (Speed vs. Quality).
3.  **Deep Learning**: Training eines BERT-Modells (Transformer).
4.  **Ensemble-Optimierung**: Auto-Tuning und Hard Example Mining.

---

In [1]:
# 1. SETUP & IMPORTS
import sys
import os
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split

# Warnungen unterdrücken für saubere Ausgabe
warnings.filterwarnings("ignore")

# Füge den 'Muster'-Ordner zum Pfad hinzu, um nlp_utils zu importieren
sys.path.append(os.path.abspath("Muster"))

# Importiere ALLES aus unserer mächtigen Bibliothek
from nlp_utilsV1 import *

print(f"✅ Setup abgeschlossen. Datenpfad: {TRAIN_DATA_PATH}")

✅ Setup abgeschlossen. Datenpfad: /Users/cristallagus/Desktop/GitHub/Verarbeitung_nat-rlicher_Sprache_Katastrophen-Tweets/Data_Set/train.csv


In [2]:
# --- KONFIGURATION ---
# Setze None für den vollen Datensatz oder eine Zahl (z.B. 500) für schnelles Testen
MAX_SAMPLES = 100#None

# 2. DATEN LADEN & SPLIT
try:
    # 1. Daten laden
    df_full = pd.read_csv(TRAIN_DATA_PATH)

    # 2. Flexible Reduzierung (Falls MAX_SAMPLES gesetzt ist)
    if MAX_SAMPLES is not None and len(df_full) > MAX_SAMPLES:
        # Wir nutzen sample mit stratify (indirekt über random_state),
        # um die Klassenverteilung beim Kürzen zu erhalten
        df_full = df_full.sample(n=MAX_SAMPLES, random_state=42).reset_index(drop=True)
        print(f"⚠️ Stichprobe aktiviert: Nutze nur {MAX_SAMPLES} Reihen (Entwicklungs-Modus).")
    else:
        print(f"✅ Voller Datensatz geladen: {df_full.shape}")

    # 3. Vorbereitung der Spalten
    if 'cleaned_text' not in df_full.columns:
        df_full['cleaned_text'] = ""

    # 4. Stratifizierter Split (80/20)
    # y muss neu definiert werden, falls gesampelt wurde
    df_train, df_val = train_test_split(
        df_full,
        test_size=0.2,
        random_state=42,
        stratify=df_full['target']
    )

    # RAM-Schonung: Original df_full löschen, wenn wir nur noch mit den Splits arbeiten
    del df_full
    import gc
    gc.collect()

    print(f"Trainings-Set: {df_train.shape}")
    print(f"Validierungs-Set: {df_val.shape}")

    # 5. df_Cleaning initialisieren (für die nachfolgende Pipeline)
    df_Cleaning = df_train.copy()

    # Deine Regel: Für jede Visualisierung/Zusammenfassung eine Tabelle zeigen
    print("\n📊 Numerische Übersicht des gewählten Trainings-Sets:")
    display(df_Cleaning['target'].value_counts().to_frame())
    display(df_Cleaning.head())

except FileNotFoundError:
    print(f"🚨 FEHLER: Datei unter {TRAIN_DATA_PATH} nicht gefunden.")
except Exception as e:
    print(f"🚨 Ein unerwarteter Fehler ist aufgetreten: {e}")

⚠️ Stichprobe aktiviert: Nutze nur 100 Reihen (Entwicklungs-Modus).
Trainings-Set: (80, 3)
Validierungs-Set: (20, 3)

📊 Numerische Übersicht des gewählten Trainings-Sets:


,count
target,
0,48
1,32


,text,target,cleaned_text
90,I totally agree. They rape kill destroy and le...,1,
83,#breakingnews Rly tragedy in MP: Some live to ...,1,
72,Don't think for one second I'm out to drown yo...,0,
46,California wildfires force thousands to evacua...,1,
45,@pattonoswalt @FoxNews Wait I thought Fecal Hu...,1,


# EDA

In [3]:
# Globale Übersicht (Ist-Zustand vor der Reinigung)
EDA_vor_reinigung(df_train)

Mancos in der Struktur (NaNs, Duplikate, Leerräume)
Mancos in der Sprache (Fremdsprachen, reine Emoji-Texte, zu kurze Texte)
Mancos im Inhalt (Slang/Kürzel, Stoppwort-Last, Vokabular-Dichte)
Mancos in der Verteilung (Wie unterscheiden sich Katastrophen von normalen Tweets?)
                🚀 TEXT-DATEN-ANALYSE (IST-ZUSTAND)

🌍 SPRACHANALYSE (Stichprobe 500 Einträge):


,Anzahl (Count),Anteil (%)
text,,
en,78,97.50
ro,1,1.25
da,1,1.25



📊 ÜBERSICHT DER SPALTEN-QUALITÄT:


,Spalte,NaN (Mancos),Unique,Kardinalität (%),Ø Zeichen,Texte m. Emojis,Texte m. Kürzeln,URLs,Sonderzeichen
0,text,0,80,100.00,99.2,7,13,39,406
1,target,0,2,2.50,-,-,-,-,-
2,cleaned_text,0,1,1.25,0.0,0,0,0,0



🧠 STRUKTURELLE TEXT-ANALYSE (KOMPLEXITÄT & NOISE):


...

,Ø Länge,Ø Großbuchst.,Texte m. Emojis
target,,,
0,92.40,6.92,1
1,109.41,8.56,6



📏 STATISTIK DER ROH-TEXTLÄNGEN:


,count,mean,std,min,25%,50%,75%,max
Zeichen-Anzahl,80.0,99.2000,36.193013,11.0,82.75,106.0,133.25,144.0
Wort-Anzahl,80.0,14.8375,6.013579,1.0,10.00,15.5,19.00,26.0



⚠️ GEFUNDENE ANOMALIEN / MANCOS:


,Anomalie-Typ,Anzahl (Count),Bedeutung
0,Leere Texte / Nur Whitespace,0,Kein Info-Gehalt
1,Extrem kurz (< 5 Zeichen),0,Kaum Kontext
2,Reine Emoji-Texte (kein Alphabet),0,Nicht sprachlich auswertbar
3,Gekürzte Begriffe (Shortforms),13,Informelle Sprache
4,Shouting (Nur Großbuchstaben),0,Extreme Emotionalität
5,Text-Duplikate (Spam-Gefahr),0,Verzerrt Vokabular
6,Inkonsistente Umbrüche (\n),5,Struktur-Rauschen


# BEREINIGUNG

In [4]:
# Standard Bereinigung (Erstellt df_Cleaning)
# Dies führt Übersetzung, Emoji-Umwandlung und Lemmatisierung durch
df_Cleaning = standard_Cleare_TEXT(df_train, ziel_sprache='en')

🚀 START NLP-PIPELINE: DF_CLEANING
DEBUG: Spalten vor Reinigung: ['text', 'target', 'cleaned_text']
🧠 Verarbeite 80 Zeilen parallel (Core-Reserve aktiv)...


NLP Pipeline:   0%|                    | 0/80 [00:00<?, ?it/s]


📊 ZUSAMMENFASSUNG DER REINIGUNG:
            Metrik  Wert
     Zeilen Vorher    80
    Zeilen Nachher    80
Duplikate entfernt     0
✅ REINIGUNG BEENDET: df_Cleaning ist bereit für Radar-Plot Signale.


# Erweiterte EDA

In [5]:
# 3. EXPLORATIVE DATENANALYSE (EDA)
# Wir nutzen nun das bereinigte DataFrame df_Cleaning für die Analyse

# 3.2 Visualisierungen
plot_target_distribution(df_Cleaning)


📊 NUMERISCHE VERTEILUNG:


,label,count,Anteil (%)
0,Keine Katastrophe,48,60.0
1,Katastrophe,32,40.0


### 📊 Interpretation: Verteilung der Zielvariable
Dieses Balkendiagramm zeigt das Verhältnis zwischen echten Katastrophen-Tweets (Rot) und normalen Tweets (Grün).
- Ein **ausgewogenes Verhältnis** ist ideal für das Training.
- Wenn eine Klasse stark dominiert (z.B. 90% Nicht-Katastrophen), muss das Modell mit Techniken wie `class_weight='balanced'` oder Resampling gegensteuern, um nicht nur die Mehrheitsklasse vorherzusagen.

In [6]:
# 3. Textlängen-Analyse (Histogramm & Violin)
# Vergleich der Zeichenanzahl zwischen Target 0 und 1
print('Verteilung der Textlängen (0 vs 1)')
plot_metric_by_target(df_Cleaning, metric_col='length')

Verteilung der Textlängen (0 vs 1)



📊 STATISTIK: length


,Status,count,mean,std,min,25%,50%,75%,max
0,Katastrophe,32.0,109.406250,27.401271,37.0,87.75,120.0,135.0,141.0
1,Keine Katastrophe,48.0,92.395833,39.854384,11.0,64.50,101.0,130.0,144.0


In [7]:
# 1. Signal-Stärke der Keywords (Top 20)
# Dies zeigt, welche Wörter am stärksten auf eine Katastrophe hindeuten
df_TEMP_KW = df_Cleaning.copy()
df_TEMP_KW['found_keyword'] = df_TEMP_KW['cleaned_text'].apply(get_hit_keyword)
print(f"📊 Analyse der Signal-Keywords")
print(f"Anzahl der analysierten Tweets: {len(df_TEMP_KW)}")
plot_feature_signal_ratio(df_TEMP_KW[df_TEMP_KW['found_keyword'].notna()],
                          feature_col='found_keyword',
                          top_n=40)
del df_TEMP_KW

📊 Analyse der Signal-Keywords
Anzahl der analysierten Tweets: 80



📊 DATENTABELLE: found_keyword


,found_keyword,mean,count
0,fire,0.833333,6


In [8]:
# 2. Geografische Signale (Top 15 Locations)
df_TEMP_LOC = df_Cleaning.copy()
df_TEMP_LOC['found_location'] = df_TEMP_LOC['cleaned_text'].apply(get_hit_location)
print('Signal-Stärke der Locations')
print(f"🌍 Analyse der geografischen Signale (aus Text)")
plot_feature_signal_ratio(df_TEMP_LOC[df_TEMP_LOC['found_location'].notna()],feature_col='found_location',top_n=15)
del df_TEMP_LOC

Signal-Stärke der Locations
🌍 Analyse der geografischen Signale (aus Text)
⚠️ Keine Daten für 'found_location' mit Count > 5 gefunden.


### 📊 Interpretation: Tweet-Länge
Dieses Histogramm vergleicht die Länge der Tweets (Anzahl der Zeichen) für beide Klassen.
- Überlappen sich die Kurven stark, ist die Länge kein gutes Unterscheidungsmerkmal.
- Oft sind Katastrophen-Tweets (Rot) etwas länger und detaillierter als alltägliche Kurznachrichten.
- Spitzen bei bestimmten Längen können auf automatisierte Bots oder Retweets hindeuten.

In [9]:
# 4. Phrasen-Analyse (Bigramme / 2-Wort-Kombinationen)
# Findet die häufigsten Wortpaare im gereinigten Text
print('Häufigste Bigramme (Wortpaare)')
plot_neutral_ngrams(df_Cleaning, n=2, top_n=15)

Häufigste Bigramme (Wortpaare)



📊 TOP 15 2-GRAMM DATEN:


,Count,N-Gram
0,2,wake anthrax
1,2,time dollar
2,2,stuart broad
3,2,service dollar
4,2,mishap dollar
5,2,lab mishap
6,2,fivesos dollar
7,2,anthrax lab
8,1,youth collision
9,1,youll glad


### 📊 Interpretation: N-Gramm Analyse (Bigramme)
Dieses Balkendiagramm zeigt die häufigsten Wortpaare (Bigramme).
- Einzelne Wörter können mehrdeutig sein ("fire" -> Kaminfeuer oder Waldbrand?).
- Bigramme liefern Kontext: "forest fire" vs. "fire place".
- Diese Analyse hilft zu verstehen, welche Phrasen starke Signale für das Modell sind.

In [10]:
plot_wordclouds(df_Cleaning)


📊 TOP 10 WÖRTER PRO KATEGORIE:


,Katastrophe,Normal
0,dollar (25),dollar (35)
1,to (14),a (20)
2,in (12),the (19)
3,of (12),i (14)
4,and (11),to (14)
5,a (11),is (11)
6,the (10),you (10)
7,i (6),my (10)
8,hundred (6),and (10)
9,wa (5),for (8)


# FEATURE ENGINEERING

In [11]:
# Wenn stop_words_combined Set oder eine Liste ist:
print("Auszug der kombinierten Stopwords:")
stop_words_combined = set(stopwords.words('english'))
print(list(stop_words_combined)[:])

# Falls du wissen willst, wie viele Wörter insgesamt gefiltert werden:
print(f"\nGesamtanzahl der Stopwords: {len(stop_words_combined)}")

Auszug der kombinierten Stopwords:
['didn', 'him', 'to', 'having', 'this', 'only', 'doesn', "they've", 'won', 'she', 'during', 'no', 'his', 'again', 'from', 'now', 'its', 's', "weren't", "mustn't", 'down', 'with', "you'll", 'themselves', 'own', 'by', 'i', "i'll", 'in', 'of', 'has', 'll', 'or', 'yours', 'be', 'same', 'the', 'some', 'am', "didn't", 'ours', 'all', 'when', "they'd", "hadn't", "i've", 'o', 'out', "she'd", 'until', 'yourself', "she'll", 'himself', 'hers', 'through', 'so', 'our', 'weren', 'don', 'they', 'are', 'such', "they'll", 'me', 'shan', 'he', "hasn't", 'her', "i'm", 'more', "he's", "couldn't", "he'd", 'on', 'doing', 'those', "wouldn't", 'here', 'you', 'into', 'wasn', 'd', "needn't", 'off', 'couldn', "shan't", "haven't", 'for', 'about', 'herself', "it'll", 'too', "shouldn't", 'mightn', 'as', 'should', 'were', 'what', 'will', 'while', 'was', "we're", 'it', 'both', 'between', 'at', 'over', 'm', 'before', 'itself', 'most', 'hadn', 'just', 'once', 'each', "we'll", 't', 'abov

In [12]:
# 5. Strategische Token-Analyse (Signal vs. Rauschen)
# Vergleicht die häufigsten Wörter in beiden Klassen und identifiziert potenzielle Stoppwörter
print('Strategische Token-Analyse (Signal vs. Rauschen)')
plot_strategic_token_analysis(df_Cleaning, top_n=30, height=600)

Strategische Token-Analyse (Signal vs. Rauschen)



🔍 STRATEGISCHE AUSWERTUNG FÜR MODELL-OPTIMIERUNG


,Typ,Tokens
0,STOPPWORT-VORSCHLAG,"two, to, you, this, dollar, it, a, are, not, t..."
1,ECHTE SIGNALE,"at, by, evacuate, four, home, hundred, let, ra..."



SIND MANUELL IM NÄCHSTEN CODE EINZUFÜGEN


In [13]:
# stop worts manuelle Anpassung
print(get_strategic_stopwords())


🛡️ SIGNAL-ENGINEERING KONFIGURATION:
| Kategorie                    |   Anzahl |
|:-----------------------------|---------:|
| Aktive Filter (Noise)        |      190 |
| Gerettete Signale (Features) |       10 |

💡 Beispiele geretteter Signale: ['after', 'are', 'as', 'at', 'by']...
{'didn', 'him', 'to', 'having', 'this', 'only', 'doesn', "they've", 'won', 'she', 'no', 'his', 'again', 'now', 'its', 's', "weren't", "mustn't", 'down', 'with', "you'll", 'themselves', 'own', 'i', "i'll", 'in', 'of', 'has', 'll', 'or', 'yours', 'be', 'same', 'the', 'some', 'am', "didn't", 'ours', 'all', 'when', "they'd", "hadn't", "i've", 'o', 'out', "she'd", 'until', 'yourself', "she'll", 'himself', 'hers', 'through', 'so', 'our', 'weren', 'don', 'they', 'such', "they'll", 'me', 'shan', 'he', "hasn't", 'her', "i'm", 'more', "he's", "couldn't", "he'd", 'on', 'doing', 'those', "wouldn't", 'here', 'you', 'wasn', 'd', "needn't", 'off', 'couldn', "shan't", 'like', "haven't", 'for', 'about', 'herself', "it'll",

In [14]:
#erstelle Engeneering mehrere neue spalten
df_Cleaning = run_full_disaster_pipeline(df_Cleaning)


🚀 STARTE ADVANCED FEATURE ENGINEERING (OS: Darwin)
🔍 Schritt 5: Kontext-Extraktion...
📝 Schritt 1: Punctuation & Caps Metrics...
📝 Berechne Subjektivität...
📝 Schritt 2: Großschreibung (Caps Count)...
🔗 Schritt 3: Source Credibility...
🎯 Schritt 4: Erzeuge Triaden-Signale (Type, Location, Time)...
📈 Schritt 7: Berechne finalen Disaster-Score...
🤖 Schritt 8: Enclave LLM Analyse (Core-Reserve aktiv)...
📦 Integrität bestätigt (80 Zeilen). Lade Schablone...

📊 ENCLAVE STATUS-BERICHT (Numerische Unterstützung):


,count
enclave_score,
1,57
0,23



📊 ENGINEERING ABGESCHLOSSEN. Numerische Übersicht für Radar-Plot:
      type_K  type_Location  type_time  type_K_L_t_combi  is_triple_signal  \
mean    0.45         0.1225     0.3375            0.0125               0.0   
min    -1.00         0.0000     0.0000            0.0000               0.0   
max     1.00         1.0000     1.0000            1.0000               0.0   

      disaster_score  is_noise_flag  
mean            1.55         0.1875  
min             0.00         0.0000  
max             8.00         1.0000  

📊 Generiere Polar-Plot basierend auf den neuen Features...



📊 SORTIERTE BASISDATEN (Präsenz-Wahrscheinlichkeit):


,caps_count_raw,dot_count,emoji_count,enclave_score,type_K,subjectivity,has_url,disaster_score,type_time,type_Location,excl_count,ques_count,is_noise_flag,is_news_style,type_K_L_t_combi,is_triple_signal
target,,,,,,,,,,,,,,,,
Keine Katastrophe,0.9167,0.6458,0.4167,0.6667,0.4375,0.6458,0.3750,0.3125,0.3125,0.1250,0.1250,0.1667,0.2500,0.0208,0.0000,0.0
Katastrophe,1.0000,0.8438,0.7812,0.7812,0.7500,0.7188,0.6562,0.5000,0.3750,0.1562,0.0938,0.0938,0.0938,0.0625,0.0312,0.0



📋 Vorschau der ersten 5 Zeilen (df_Cleaning):


# ML Auswahl und endscheidung
Für den datensatz welche modelle und rechen geschwindigkeiten jedes modell Hatt das in frage kommt

In [15]:
# 4. MODELL-BENCHMARK (SPEED VS QUALITY)
# Wir lassen 25+ Modelle gegeneinander antreten, um den besten Kandidaten zu finden.

# Vorbereitung der Daten für den Benchmark (X und y trennen)
# Wir nutzen hier df_Cleaning, da es die qualitativ hochwertigeren Daten enthält
X_train_bench = df_Cleaning[['text', 'cleaned_text']].copy()
y_train_bench = df_Cleaning['target']

X_val_bench = df_val.copy()
# Für Validation müssen wir auch kurz cleanen (für Benchmark Features)
# Hinweis: Für einen fairen Vergleich sollte man idealerweise auch hier standard_Cleare_TEXT nutzen,
# aber für den schnellen Benchmark reicht oft der einfache Cleaner.
cleaner = TextCleaner(stopwords.words('english'))
X_val_bench.loc[:, 'cleaned_text'] = cleaner.transform(X_val_bench['text'])
X_val_bench = X_val_bench[['text', 'cleaned_text']]
y_val_bench = df_val['target']

print("Starte Benchmark...")
benchmark_results = run_comprehensive_benchmark(X_train_bench, y_train_bench, X_val_bench, y_val_bench, stopwords.words('english'))

Starte Benchmark...
### START: Benchmark ###
### System-Check ###
OS: Darwin 24.6.0
CPU Kerne: 8
RAM: 16.00 GB
✅ GPU gefunden: 1x [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

⏱️ Starte Effizienz-Check (Worst-Case Szenario)...



Starte vollen Benchmark (Warnungen werden unterdrückt)...


Benchmarking:   0%|                    | 0/29 [00:00<?, ?it/s]


⚠️ 2 Warnungen wurden während des Benchmarks unterdrückt.
           Model  Training Time (s)  Prediction Time (s)   Recall  F1-Score  Accuracy  Precision
             MLP           0.025506             0.002966 0.645833  0.642857      0.70   0.718750
             LDA           0.019855             0.007292 0.625000  0.626667      0.65   0.631868
   Random Forest           0.120864             0.016789 0.625000  0.600000      0.70   0.833333
     Extra Trees           0.093503             0.016606 0.625000  0.600000      0.70   0.833333
Nearest Centroid           0.005021             0.002120 0.625000  0.600000      0.60   0.625000
      Perceptron           0.008226             0.002550 0.604167  0.595960      0.60   0.600000
       SVC (RBF)           0.005903             0.001951 0.562500  0.548872      0.55   0.560606
      Extra Tree           0.008199             0.002601 0.562500  0.498208      0.65   0.815789
         Bagging           0.031111             0.016875 0.562500  0


📊 BENCHMARK DATEN (TOP 10 NACH Recall):


,Model,Recall,F1-Score,Accuracy,Training Time (s)
28,MLP,0.6458,0.6429,0.70,0.0255
25,LDA,0.6250,0.6267,0.65,0.0199
16,Random Forest,0.6250,0.6000,0.70,0.1209
17,Extra Trees,0.6250,0.6000,0.70,0.0935
23,Nearest Centroid,0.6250,0.6000,0.60,0.0050
5,Perceptron,0.6042,0.5960,0.60,0.0082
12,SVC (RBF),0.5625,0.5489,0.55,0.0059
15,Extra Tree,0.5625,0.4982,0.65,0.0082
18,Bagging,0.5625,0.4982,0.65,0.0311
19,AdaBoost,0.5625,0.4982,0.65,0.0533


### 📊 Interpretation: Modell-Benchmark (Speed vs. Quality)
Dieser Scatterplot ist entscheidend für die Modellauswahl.
- **X-Achse (Log-Skala)**: Trainingszeit in Sekunden. Je weiter links, desto schneller.
- **Y-Achse**: F1-Score (Qualität). Je höher, desto besser.
- **Der "Sweet Spot"**: Modelle oben links (schnell & gut).
- Oft sind **Lineare Modelle** (Logistic Regression, SGD) extrem schnell und überraschend gut.
- **Transformer (BERT)** wären ganz oben, aber sehr weit rechts (langsam).
- Dieser Plot hilft zu entscheiden: Brauche ich maximale Performance (BERT) oder reicht ein schnelles Modell für Echtzeit-Anwendungen?

In [16]:
# 5. HAUPT-ANALYSE (ENSEMBLE + AUTO-TUNING)
# Basierend auf dem Benchmark nutzen wir nun die optimierte Pipeline (LogReg + RF + MLP)
# inklusive Hard Example Mining und Threshold-Optimierung.

best_ensemble = run_analysis(X_train_bench, y_train_bench, X_val_bench, y_val_bench, stopwords.words('english'))

### START: Analyse Pipeline (Recall-Optimiert) ###


Features:   0%|                    | 0/80 [00:00<?, ?it/s]

Features:   0%|                    | 0/20 [00:00<?, ?it/s]


🤖 Auto-Tuning (Fokus: Recall)...


GridSearch:   0%|                    | 0/24 [00:00<?, ?it/s]

...

,Zone,Durchschn. Länge,Anzahl Tweets
0,Sicher Normal,105.428571,14
1,Unsicher (Grauzone),127.400000,5
2,Sicher Katastrophe,136.000000,1



📊 FEHLER-STATISTIK & KENNZAHLEN:


,Fehlertyp,Anzahl,Ø Wahrscheinlichkeit,Ø Subjektivität
0,Falscher Alarm (FP),1,0.577,0.00
1,Verpasste Katastrophe (FN),4,0.295,0.15



📊 NUMERISCHE ZUSAMMENFASSUNG:


,Modell,Optimal Threshold,F1-Score,Recall,Accuracy
0,Voting Ensemble (Optimiert),0.44,0.7151,0.7083,0.75


### 📊 Interpretation: Konfusionsmatrix & ROC-Kurve
**Konfusionsmatrix (Links):**
- Zeigt, wo das Modell Fehler macht.
- **Oben rechts (False Positive)**: Falscher Alarm (Normaler Tweet als Katastrophe erkannt).
- **Unten links (False Negative)**: Verpasste Gefahr (Katastrophe als normal erkannt) -> **Das wollen wir vermeiden!**

**Trennschärfe (Rechts):**
- Zeigt die Wahrscheinlichkeitsverteilung.
- Idealerweise sind die roten Balken (Katastrophen) ganz rechts bei 1.0 und die grünen ganz links bei 0.0.
- Der gestrichelte Strich ist der **optimale Threshold**, ab dem Alarm geschlagen wird.

**ROC-Kurve (Unten):**
- Zeigt das Verhältnis von Trefferquote zu Fehlalarmquote.
- Eine Kurve nahe der oberen linken Ecke (AUC nahe 1.0) ist perfekt.
- Die Diagonale wäre reines Raten (AUC 0.5).

In [17]:
# 6. DEEP LEARNING (BERT)
# Wir trainieren ein DistilBERT-Modell für maximale Genauigkeit.

# Für BERT brauchen wir die Rohdaten (X_train['text']), da der Tokenizer im Modell steckt.
# Wir nutzen hier df_Cleaning['text'] (bereinigt um Emojis/Shorts) oder df_train['text'] (Rohdaten).
# BERT profitiert oft von leicht bereinigten, aber nicht lemmatisierten Daten.
bert_model = run_bert_training(df_Cleaning['text'], df_Cleaning['target'], df_val['text'], df_val['target'])

### START: BERT Pipeline ###
### System-Check ###
OS: Darwin 24.6.0
CPU Kerne: 8
RAM: 16.00 GB
✅ GPU gefunden: 1x [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2026-01-06 23:47:40.975623: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2026-01-06 23:47:40.975669: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2026-01-06 23:47:40.975675: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
I0000 00:00:1767739660.975694 1377377 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1767739660.975725 1377377 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2026-01-06 23:47:44.557156: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


              precision    recall  f1-score   support

           0       1.00      0.83      0.91        12
           1       0.80      1.00      0.89         8

    accuracy                           0.90        20
   macro avg       0.90      0.92      0.90        20
weighted avg       0.92      0.90      0.90        20




📊 NUMERISCHE MATRIX & METRIKEN:


,Vorhergesagt: Normal,Vorhergesagt: Katastrophe
Tatsächlich: Normal,10,2
Tatsächlich: Katastrophe,0,8



📊 ROC-KURVEN STÜTZPUNKTE (Auszug):


,False Positive Rate,True Positive Rate,Threshold
0,0.0000,0.000,inf
0,0.0000,0.000,inf
1,0.0000,0.125,0.9998
1,0.0000,0.125,0.9998
2,0.0000,0.875,0.9967
2,0.0000,0.875,0.9967
3,0.0833,0.875,0.9678
3,0.0833,0.875,0.9678
4,0.0833,1.000,0.9657
5,1.0000,1.000,0.0053


### 📊 Interpretation: BERT Trainingsverlauf & Ergebnisse
**Trainingsverlauf:**
- Zeigt Loss (Fehler) und Accuracy über die Epochen.
- **Wichtig**: Wenn der "Validation Loss" steigt, während der "Training Loss" sinkt, beginnt das Modell auswendig zu lernen (**Overfitting**). Early Stopping verhindert das.

**BERT Performance:**
- BERT versteht den Kontext von Wörtern (z.B. Sarkasmus) oft besser als klassische Modelle.
- Vergleichen Sie die Konfusionsmatrix hier mit der des Ensembles. Hat BERT weniger "Verpasste Gefahren" (False Negatives)?

In [18]:
# 7. FEATURE IMPORTANCE
# Wir schauen uns an, welche Wörter für das Ensemble-Modell am wichtigsten waren.
# Wir extrahieren den LogReg-Teil aus dem Ensemble für die Visualisierung
try:
    # Zugriff auf den VotingClassifier -> Estimators -> LogisticRegression
    logreg = best_ensemble.named_steps['clf'].estimators_[0]
    # Zugriff auf den TfidfVectorizer im Preprocessor
    tfidf = best_ensemble.named_steps['prep'].transformers_[0][1]
    feature_names = tfidf.get_feature_names_out()
    
    plot_feature_importance(logreg, feature_names, top_n=20)
except Exception as e:
    print(f"Feature Importance konnte nicht erstellt werden: {e}")


📊 TOP 20 FEATURE-WERTE (Numerische Daten):


,Feature,Importance
659,mom,0.6945
1202,wtf,0.6945
1203,wtf mom,0.6945
289,drowned child,0.6945
660,mom drowned,0.6945
1513,Eng_Feature_1514,0.6243
190,child,0.5992
288,drowned,0.5633
1442,Eng_Feature_1443,0.4551
1364,Eng_Feature_1365,0.4551


### 📊 Interpretation: Feature Importance
Dieser Plot zeigt, welche Wörter das Modell am stärksten beeinflussen.
- **Blaue Balken (Positiv)**: Wörter, die stark für "Katastrophe" sprechen (z.B. "hiroshima", "wildfire").
- **Rote Balken (Negativ)**: Wörter, die stark gegen eine Katastrophe sprechen.
- **Plausibilitäts-Check**: Sind hier Wörter dabei, die keinen Sinn ergeben (z.B. "the", "and")? Falls ja, muss die Stoppwort-Liste erweitert werden.

# 8. FAZIT & PHILOSOPHIE DES PROJEKTS
### Zusammenfassung der Ergebnisse

Wir haben eine umfassende Analyse von Katastrophen-Tweets durchgeführt, beginnend mit einer detaillierten explorativen Datenanalyse bis hin zum Training modernster Deep-Learning-Modelle.

**Wichtige Erkenntnisse:**
1.  **Datenqualität**: Die Bereinigung (Entfernen von URLs, Emojis-Handling) war entscheidend, um das Rauschen zu reduzieren.
2.  **Modell-Vergleich**: Der Benchmark zeigte, dass einfache Modelle wie Logistic Regression extrem schnell und solide sind, aber komplexe Modelle (Ensembles, BERT) bei der Feinheit (F1-Score) oft die Nase vorn haben.
3.  **Deep Learning**: Das BERT-Modell konnte durch sein Kontextverständnis oft Nuancen erkennen, die klassischen Modellen entgingen (z.B. Sarkasmus oder metaphorische Verwendung von "fire").

### Philosophie & Wiederverwendbarkeit für die Zukunft

Dieses Projekt wurde bewusst so strukturiert, dass es als **Blaupause für zukünftige NLP-Projekte** dient. Die Kernidee ist die Trennung von Ausführung (dieses Notebook) und Logik (die `nlp_utils.py` Bibliothek).

**Vorteile dieses Ansatzes:**
- **Vollständigkeit & Korrektheit**: Alle wichtigen Funktionen – von der EDA über den Modell-Benchmark bis zur Visualisierung – sind zentral in `nlp_utils.py` gesammelt. Dadurch wird sichergestellt, dass bei neuen Projekten keine wichtigen Analyseschritte vergessen werden. Es ist eine Checkliste in Codeform.
- **Wiederverwendbarkeit**: Für ein neues Projekt (z.B. Spam-Erkennung) muss nur die globale Konfiguration in `nlp_utils.py` (Pfade, Spaltennamen) angepasst werden. Die `run_comprehensive_benchmark` Funktion kann sofort auf die neuen Daten angewendet werden, um schnell das beste Modell zu finden.
- **Wissensspeicher**: Die Bibliothek dient als wachsender Wissensspeicher. Jede neue Funktion, jedes bessere Modell und jede neue Visualisierung, die wir entwickeln, wird dort hinzugefügt. So geht kein Code verloren und zukünftige Projekte profitieren automatisch vom gesammelten Wissen.
- **Sauberkeit & Fokus**: Das Notebook bleibt sauber, lesbar und fokussiert sich auf die Interpretation der Ergebnisse, während die komplexe Implementierung in der `.py`-Datei gekapselt ist.

Dieser Aufbau garantiert, dass wir bei jedem neuen Projekt auf einem soliden, vollständigen und professionellen Fundament aufbauen.
